In [1]:
!pip install mlflow keras-tuner optuna dvc -q
!pip install dvc[s3]
!apt install git -y
!pip install git+https://github.com/philipperemy/keras-tcn.git -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.keras
import os
import subprocess
import optuna
from tcn import TCN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

In [ ]:
MLFLOW_TRACKING_URI = ""
EXPERIMENT_NAME = "AQI hyperparameter & model testing (Manual)"
WINDOW_SIZE = 24
PREDICT_HORIZON = 72  # Predict next 3 days = 72 hours

In [4]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/home/umair/mlruns/3', creation_time=1752022350617, experiment_id='3', last_update_time=1752022350617, lifecycle_stage='active', name='AQI hyperparameter & model testing (Manual)', tags={}>

In [5]:
!git init
!dvc init
!dvc remote add -d myremote s3://s3-bucket-umairrr

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                  

In [6]:
!git clone https://github.com/uma1r111/10pearls-AQI-Project-
%cd 10pearls-AQI-Project-

Cloning into '10pearls-AQI-Project-'...
remote: Enumerating objects: 410, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 410 (delta 84), reused 148 (delta 50), pack-reused 224 (from 1)
Receiving objects: 100% (410/410), 5.60 MiB | 18.45 MiB/s, done.
Resolving deltas: 100% (192/192), done.
/content/10pearls-AQI-Project-


In [ ]:
# Load and preprocess data
print("\nPulling latest feature_selection.csv from DVC remote (S3)...")
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = ""
subprocess.run(["dvc", "pull"], check=True)


Pulling latest feature_selection.csv from DVC remote (S3)...


CompletedProcess(args=['dvc', 'pull'], returncode=0)

In [8]:
for root, dirs, files in os.walk(".", topdown=True):
    for name in files:
        print(os.path.join(root, name))

./README.md
./.dvcignore
./requirements.txt
./.gitignore
./full_preprocessed_aqi_weather_data_with_all_features.csv
./karachi_weather_apr1_to_current.csv
./fetch_daily_data.py
./feature_selection.csv.dvc
./feature_selection.csv
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.ipynb
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.txt
./.dvc/config
./.dvc/.gitignore
./.dvc/cache/files/md5/a3/42be088d6bbebcb8002a166df6f909
./.dvc/tmp/btime
./.dvc/tmp/lock
./Data Preprocessing/Data_PreProcessing.ipynb
./Data Preprocessing/data_quality_check.py
./Data Preprocessing/run_preprocessing.py
./Data Collection/karachi_weather_jun2025.ipynb
./Data Collection/pollutants info Apr - Jun.ipynb
./Data Collection/karachi_weather_apr1_jun14.ipynb
./.github/workflows/feature_engineering.yml
./.github/workflows/feature_selection.yml
./.github/workflows/update_data.yml
./Feature Selection/initial_feature_extraction.ipynb
./Feature Selection/feature_selection.py


In [9]:
df = pd.read_csv("feature_selection.csv")

In [10]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df[(df["datetime"] >= "2025-04-01") & (df["datetime"] <= "2025-07-07")]
df = df.sort_values("datetime")

features = df.drop(columns=["datetime"])
num_features = features.shape[1]  # total features including AQI

In [11]:
print(df["datetime"].head())
print("Filtered Rows:", len(df))

0   2025-04-01 00:00:00
1   2025-04-01 01:00:00
2   2025-04-01 02:00:00
3   2025-04-01 03:00:00
4   2025-04-01 04:00:00
Name: datetime, dtype: datetime64[ns]
Filtered Rows: 2329


# **Sequence Building for Walk forward Validation**

In [12]:
def create_sequences(X, window_size, horizon):
    Xs, ys = [], []
    for i in range(len(X) - window_size - horizon):
        Xs.append(X[i:(i + window_size)].values)
        ys.append(X[(i + window_size):(i + window_size + horizon)].values)
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(features, WINDOW_SIZE, PREDICT_HORIZON)
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

In [13]:
print("Total Samples:", len(features))
print("X_seq shape:", X_seq.shape)
print("y_seq shape:", y_seq.shape)

Total Samples: 2329
X_seq shape: (2233, 24, 13)
y_seq shape: (2233, 72, 13)


# **Optuna Hyperparameter Tuning**

In [14]:
def optuna_tcn_objective(trial):
    nb_filters = trial.suggest_int('nb_filters', 32, 128, step=32)
    kernel_size = trial.suggest_categorical('kernel_size', [2, 3, 4])
    nb_stacks = trial.suggest_int('nb_stacks', 1, 3)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4, step=0.1)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    epochs = trial.suggest_int("epochs", 30, 60)
    batch_size = trial.suggest_categorical("batch_size", [32, 64])

    model = Sequential()
    model.add(TCN(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        nb_filters=nb_filters,
        kernel_size=kernel_size,
        nb_stacks=nb_stacks,
        dilations=[1, 2, 4, 8],
        activation=activation,
        dropout_rate=dropout_rate,
        use_skip_connections=True
    ))
    model.add(Dense(PREDICT_HORIZON * num_features))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    model.fit(
        X_train, y_train.reshape(y_train.shape[0], -1),
        validation_data=(X_val, y_val.reshape(y_val.shape[0], -1)),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=0
    )

    preds = model.predict(X_val).reshape(y_val.shape)
    rmse = sqrt(mean_squared_error(y_val.flatten(), preds.flatten()))
    return rmse

# Run Optuna search
study = optuna.create_study(direction='minimize')
study.optimize(optuna_tcn_objective, n_trials=20)

# Log best trial to MLflow
with mlflow.start_run(run_name="Best_TCN_Keras_Hyperparams"):
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)
    print("Best parameters:", study.best_params)

[I 2025-07-10 11:14:21,619] A new study created in memory with name: no-name-46c63fd1-6410-47f3-8521-5a9898b7cf3c
/usr/local/lib/python3.11/dist-packages/tcn/tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step


[I 2025-07-10 11:15:04,309] Trial 0 finished with value: 37.17428480251352 and parameters: {'nb_filters': 32, 'kernel_size': 2, 'nb_stacks': 2, 'dropout_rate': 0.1, 'activation': 'tanh', 'learning_rate': 3.3651301024930325e-05, 'epochs': 33, 'batch_size': 32}. Best is trial 0 with value: 37.17428480251352.
/usr/local/lib/python3.11/dist-packages/tcn/tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step


[I 2025-07-10 11:15:47,101] Trial 1 finished with value: 9.781488996628852 and parameters: {'nb_filters': 96, 'kernel_size': 2, 'nb_stacks': 3, 'dropout_rate': 0.30000000000000004, 'activation': 'tanh', 'learning_rate': 0.00751450278577543, 'epochs': 30, 'batch_size': 64}. Best is trial 1 with value: 9.781488996628852.
/usr/local/lib/python3.11/dist-packages/tcn/tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


...

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step


[I 2025-07-10 11:49:16,204] Trial 18 finished with value: 33.76161084378452 and parameters: {'nb_filters': 96, 'kernel_size': 4, 'nb_stacks': 2, 'dropout_rate': 0.1, 'activation': 'relu', 'learning_rate': 1.2803421788887197e-05, 'epochs': 58, 'batch_size': 64}. Best is trial 15 with value: 6.587638762916589.
/usr/local/lib/python3.11/dist-packages/tcn/tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


[I 2025-07-10 11:49:45,775] Trial 19 finished with value: 7.380537661677128 and parameters: {'nb_filters': 32, 'kernel_size': 4, 'nb_stacks': 1, 'dropout_rate': 0.1, 'activation': 'relu', 'learning_rate': 0.00020861974992693528, 'epochs': 45, 'batch_size': 64}. Best is trial 15 with value: 6.587638762916589.


Best parameters: {'nb_filters': 64, 'kernel_size': 4, 'nb_stacks': 1, 'dropout_rate': 0.1, 'activation': 'relu', 'learning_rate': 0.00014334240518640865, 'epochs': 50, 'batch_size': 64}
🏃 View run Best_TCN_Keras_Hyperparams at: http://172.174.154.85:8000/#/experiments/3/runs/6d0da9bc10c24c7d800907ffb028d3df
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **TCN Model Fit**

In [15]:
best_params = study.best_params
with mlflow.start_run(run_name="Final_TCN_Model_Optuna"):
    model = Sequential()
    model.add(TCN(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        nb_filters=best_params['nb_filters'],
        kernel_size=best_params['kernel_size'],
        nb_stacks=best_params['nb_stacks'],
        dilations=[1, 2, 4, 8],
        activation=best_params['activation'],
        dropout_rate=best_params['dropout_rate'],
        use_skip_connections=True
    ))
    model.add(Dense(PREDICT_HORIZON * num_features))
    model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mse')

    model.fit(
        X_train, y_train.reshape(y_train.shape[0], -1),
        validation_data=(X_val, y_val.reshape(y_val.shape[0], -1)),
        epochs=best_params['epochs'],
        batch_size=best_params['batch_size'],
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )

    preds = model.predict(X_val).reshape(y_val.shape)
    rmse = sqrt(mean_squared_error(y_val.flatten(), preds.flatten()))
    mae = mean_absolute_error(y_val.flatten(), preds.flatten())

    mlflow.log_params(best_params)
    mlflow.log_metric("final_rmse", rmse)
    mlflow.log_metric("final_mae", mae)
    mlflow.set_tag("model_type", "TCN_Optuna")
    mlflow.set_tag("train_start", "2025-04-01")
    mlflow.set_tag("train_end", "2025-07-07")
    mlflow.keras.log_model(model, artifact_path="model")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/tcn/tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 7063.4614 - val_loss: 1937.2861
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 1928.9902 - val_loss: 1306.5500
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 1324.4336 - val_loss: 811.6910
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 890.3195 - val_loss: 430.0711
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 589.7610 - val_loss: 236.3802
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 434.4352 - val_loss: 154.5330
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 341.0075 - val_loss: 125.8999
Epoch 8/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 298.5884 - val_loss: 107.8654
Epoch 9/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 270.2211 - val_loss: 92.5456
Epoch 10/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 258.5970 - val_loss: 85.6133
Epoch 11/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 242.1798 - val_loss: 76.3388
Epoch 12/50
28/28 ━━━━━━━━━━

2025/07/10 11:51:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/10 11:51:20 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/07/10 11:51:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Final_TCN_Model_Optuna at: http://172.174.154.85:8000/#/experiments/3/runs/7bbb312ea37b467c8d41ef2d61e4e3d5
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **Prediction**

In [16]:
last_seq = features.values[-WINDOW_SIZE:].reshape(1, WINDOW_SIZE, num_features)
pred_next = model.predict(last_seq).reshape(PREDICT_HORIZON, num_features)
future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
                              periods=PREDICT_HORIZON, freq='H')

future_df = pd.DataFrame(pred_next, columns=features.columns)
future_df["datetime"] = future_dates
future_df.to_csv("future_predictions_tcn_optuna.csv", index=False)
mlflow.log_artifact("future_predictions_tcn_optuna.csv")

print(future_df.head())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
   aqi_us_lag1  aqi_us_lag12  aqi_us_lag24      pm2_5  log_pm10  \
0    85.479881     84.552452     83.521080  27.228628  7.231478   
1    86.091522     86.374741     87.600517  29.518707  6.707943   
2    88.006607     88.077278     87.764664  30.564623  4.600166   
3    88.409004     88.593880     88.769470  27.725271  4.978996   
4    87.022896     86.598976     84.185013  28.340736  4.542718   

   scaled_humidity_%  scaled_temp_C_scaled_log_windspeed_kph   log_so2  \
0          -1.454342                               -0.412933  3.079708   
1          -0.699970                                0.826961  0.501684   
2          -0.198372                                1.372918  2.956789   
3          -0.626525                                1.411800  2.675304   
4           0.897943                               -0.388300  2.053025   

   day_of_week  scaled_temp_C  scaled_temp_C_scaled_o3   log_no2     aqi_us  \
0     3.149564       1.863293      

/tmp/ipython-input-16-3151023325.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
